# Analyse et visualisation de données avec Python
## Créer des graphiques avec Altair
Questions
* Comment faire davantage de visualisation en Python?
* Comment faire des graphiques modernes et interactifs?
* Qu'est-ce que la "grammaire des graphiques"?

Objectifs
* Créer un objet `alt.Chart`.
* Créer différents types de graphiques.
* Modifier l'apparence du graphique:
  * Configurer les couleurs.
  * Éditer le titre et le nom des axes.
* Diviser une figure en facettes.
* Sauvegarder une figure en image et en version interactive.

In [ ]:
import numpy as np
import pandas as pd

# Charger et nettoyer les données
surveys_complet = pd.read_csv('../data/surveys.csv')
surveys_complet = surveys_complet.dropna()
surveys_complet

## Pourquoi `altair`? Pourquoi pas `matplotlib`?
Bien que `matplotlib` soit un module de visualisation largement
répandu, très flexible et puissant, son utilisation est parfois
compliquée.

Pour ce chapitre, nous allons utiliser le module `altair` qui
[facilite la création de graphiques hautement informatifs](https://altair-viz.github.io/index.html)
tout en s'intégrant bien avec Pandas. Le fonctionnement de `altair`
se base sur la grammaire des graphiques interactifs de
[Vega-Lite](https://vega.github.io/vega-lite/).

In [ ]:
import altair as alt

Étant donné que les graphiques générés par Altair ne sont pas que des
images statiques, l'information générée est parfois très lourde et
elle s'accumule dans le notebook s'il y a plusieurs graphiques.
Néanmoins, pour contourner la limite de 5000 rangées dans le DataFrame
de données, on peut désactiver cette limite, à nos risques.

In [ ]:
alt.data_transformers.disable_max_rows()

## Générer des graphiques avec `altair`
Les graphiques `altair` sont construits étape par étape à partir
d'un objet de type `Chart`:
* La première méthode obligatoire débute par `mark_`. Par exemple,
  `mark_point()`. À ce stade, c'est normal que tout soit concentré
  en un point.

In [ ]:
# Création de l'objet Chart et choix du type de graphique
alt.Chart(surveys_complet).mark_point()

* La prochaine étape consiste à
  [encoder](https://altair-viz.github.io/user_guide/encoding.html)
  des canaux liant certaines variables du DataFrame à divers éléments
  du graphiques. Les principaux paramètres de `encode()` sont :
  `x`, `y`, `color`, `shape` et `size`.

In [ ]:
# Définition des axes ; les points prennent leur position
alt.Chart(surveys_complet).mark_point().encode(
    x='hindfoot_length',
    y='weight',
)

* On peut ensuite rendre le graphique interactif, ce qui permet
  de naviguer dans le graphique à l'aide de la souris.

In [ ]:
# Permettre les interactions avec la souris
alt.Chart(surveys_complet).mark_point().encode(
    x='hindfoot_length',
    y='weight',
).interactive()

* Enfin, on peut faire afficher certaines variables pour chaque point
  lorsqu'on passe le pointeur de souris.

In [ ]:
# Ajouter du bruit aux longueurs et aux poids
surveys_complet['longueur_bruitee'] = (
    surveys_complet['hindfoot_length'] +
    np.random.uniform(-0.5, 0.5, len(surveys_complet))
)
surveys_complet['poids_bruite'] = (
    surveys_complet['weight'] +
    np.random.uniform(-0.5, 0.5, len(surveys_complet))
)

# Permettre de voir les valeurs associées aux points
alt.Chart(surveys_complet).mark_point().encode(
    x='longueur_bruitee',
    y='poids_bruite',
    tooltip=['plot_id', 'species_id', 'hindfoot_length', 'weight'],
).interactive()

* Dans les faits, chaque graphique est une spécification en format JSON.

In [ ]:
# Sauvegarder l'objet Chart dans une variable
graphique = alt.Chart(surveys_complet).mark_point().encode(
    x='longueur_bruitee',
    y='poids_bruite',
    tooltip=['plot_id', 'species_id', 'hindfoot_length', 'weight'],
).interactive()

# Obtenir le code JSON et séparer par ligne de texte
lignes_json = graphique.to_json().split('\n')

print('\n'.join(lignes_json[:24]))
print(f'\n... ({len(lignes_json) - 48} autres lignes de JSON)\n')
print('\n'.join(lignes_json[-24:]))

* Après la création du graphiques, c'est possible de le sauvegarder dans le format de notre choix.

In [ ]:
graphique.save("scatterplot.html")

In [ ]:
try:
    graphique.save("scatterplot.png")
except BaseException as err:
    print('Erreur:', err)
    print('-> Il vaut mieux utiliser le bouton (...)')

### Exercice - Créer un histogramme
À partir du DataFrame `surveys_complet`, faites afficher le
décompte du nombre d'enregistrements pour chaque `plot_id`. Indices :
* Utilisez
  [`mark_bar()`](https://altair-viz.github.io/gallery/simple_bar_chart.html)
  pour générer
  [l'histogramme](https://altair-viz.github.io/gallery/simple_histogram.html)
* Sur l'axe horizontal, spécifiez la variable `'plot_id'`
* Sur l'axe vertical, spécifiez `'count()'` pour que Altair fasse
  automatiquement le décompte, ce qui évite de passer par la méthode
  `groupby()` du DataFrame

In [ ]:
alt.Chart(surveys_complet).mark_bar().encode(
    x='plot_id',
    y='count()'
)

## Construire un graphique par étapes
* Rappel : les éléments de base de tout graphique Altair sont
  le `Chart()` avec le DataFrame, un type de graphique `mark_*()`
  et des variables utilisées dans `encode()`.
* Ensuite, on commence à modifier le graphique pour en extraire davantage d'information. Par exemple, avec de la transparence (`alpha`) :

In [ ]:
(p9.ggplot(data=surveys_complet,
           mapping=p9.aes(x='hindfoot_length', y='weight'))
    + p9.geom_point(alpha=0.05)
)

* Une couleur pour chaque type de d'espèce; il faut "mapper" la variable `species_id` à l'`aes` `color` :

In [ ]:
(p9.ggplot(data=surveys_complet,
           mapping=p9.aes(x='hindfoot_length', y='weight', color='species_id'))
    + p9.geom_point(alpha=0.05)
    + p9.guides(colour=p9.guide_legend(override_aes={"alpha": 1.0}))
)

* Modifier le nom des axes :

In [ ]:
(p9.ggplot(data=surveys_complet,
           mapping=p9.aes(x='hindfoot_length', y='weight', color='species_id'))
    + p9.geom_point(alpha=0.05)
    + p9.guides(colour=p9.guide_legend(override_aes={"alpha": 1.0}))
    + p9.xlab("Longueur de patte (mm)")
    + p9.ylab("Poids (g)")
    + p9.ggtitle("Poids selon la longueur de patte")
)

* Utiliser un axe semi-log :

In [ ]:
(p9.ggplot(data=surveys_complet,
           mapping=p9.aes(x='hindfoot_length', y='weight', color='species_id'))
    + p9.geom_point(alpha=0.05)
    + p9.guides(colour=p9.guide_legend(override_aes={"alpha": 1.0}))
    + p9.xlab("Longueur de patte (mm)")
    + p9.ylab("Poids (g)")
    + p9.ggtitle("Poids selon la longueur de patte")
    + p9.scale_y_log10()
)

* Changer le thème (`theme_*`) ou des éléments spécifiques du thème :
  * pour avoir un arrière-plan en noir et blanc, on utiliserait le
    thème prédéfini `theme_bw()`;
  * pour modifier la taille du texte, on peut créer un thème `theme()`
    et modifier une de ses propriétés.

In [ ]:
(p9.ggplot(data=surveys_complet,
           mapping=p9.aes(x='hindfoot_length', y='weight', color='species_id'))
    + p9.geom_point(alpha=0.05)
    + p9.guides(colour=p9.guide_legend(override_aes={"alpha": 1.0}))
    + p9.xlab("Longueur de patte (mm)")
    + p9.ylab("Poids (g)")
    + p9.ggtitle("Poids selon la longueur de patte")
    + p9.scale_y_log10()
    + p9.theme_bw()
    + p9.theme(legend_position="top")  # bottom, left, right
)

### Exercice - Modifier le `bar`-plot
Adaptez le `bar`-plot de l'exercice précédent pour associer la variable `sex` au paramètre de couleur `fill`. Spécifiez ensuite une liste de couleurs (`"blue"` et `"orange"`) via la fonction `scale_fill_manual()` (voir la [référence de l'API](https://plotnine.readthedocs.io/en/stable/api.html#color-and-fill-scales) pour plus d'information) :

In [ ]:
(p9.ggplot(data=surveys_complet,
           mapping=p9.aes(x='plot_id', fill='sex'))
    + p9.geom_bar()
    + p9.scale_fill_manual(["blue", "orange"])
)

## Visualiser des distributions
* Un box-plot peut être utilisé :

In [ ]:
(p9.ggplot(data=surveys_complet,
           mapping=p9.aes(x='species_id',
                          y='weight'))
    + p9.geom_boxplot()
    + p9.scale_y_log10()
)

* On peut ajouter un nuage de points verts derrière le box-plot :

In [ ]:
(p9.ggplot(data=surveys_complet,
           mapping=p9.aes(x='species_id',
                          y='weight'))
    + p9.geom_jitter(alpha=0.1, color="green")
    + p9.geom_boxplot(alpha=0)
    + p9.scale_y_log10()
)

### Exercice - Distributions
* Affichez un **violin-plot** transparent par-dessus les points et forcez la couleur de ligne `"grey"`.
* Pour l'axe des `x`, on veut que les différentes valeurs numériques de `plot_id` soient considérées comme des catégories. Pour ce faire, on utilisera `'factor(plot_id)'`.
* Faites en sorte que la couleur des points soit en fonction de l'identifiant d'espèce `'species_id'`.

In [ ]:
(p9.ggplot(data=surveys_complet,
           mapping=p9.aes(x='factor(plot_id)',
                          y='weight',
                          color='species_id'))
    + p9.geom_jitter(alpha=0.3)
    + p9.geom_violin(alpha=0, color="grey")
    + p9.scale_y_log10()
)

## Visualiser des données selon le temps
* Calculez le nombre d'enregistrements par type d'espèces pour chaque année.
* Réinitialisez l'index ; `year` et `species_id` deviendront des colonnes.

In [ ]:
yearly_counts = surveys_complet.groupby(['year', 'species_id'])['species_id'].count()
yearly_counts = yearly_counts.reset_index(name='counts')
yearly_counts

* La visualisation peut ensuite se faire avec un "line-plot" (ou `geom_line`) avec les années en `x` et les décomptes en `y`.
* Afin d'avoir une ligne par espèce, il faut spécifier l'option couleur selon l'identifiant d'espèce.

In [ ]:
(p9.ggplot(data=yearly_counts,
           mapping=p9.aes(x='year',
                          y='counts',
                          color='species_id'))
    + p9.geom_line()
)

## Création de facettes
* `plotnine` a une technique spéciale appelée *création de facettes* permettant de diviser un graphique en plusieurs sous-graphiques selon une variable de type catégorie.

In [ ]:
(p9.ggplot(data=surveys_complet,
           mapping=p9.aes(x='hindfoot_length',
                          y='weight',
                          color='species_id'))
    + p9.geom_point(alpha=0.1)
    + p9.facet_wrap('sex')
)

* Avec `facet_wrap()`, nul besoin de convertir en `factor`:

In [ ]:
(p9.ggplot(data=surveys_complet,
           mapping=p9.aes(x='hindfoot_length',
                          y='weight',
                          color='species_id'))
    + p9.geom_point(alpha=0.1)
    + p9.facet_wrap('plot_id')
)

* La fonction `facet_grid()` permet de spécifier l'arrangement d'une grille de graphiques avec la notation `rangées ~ colonnes` de graphiques.

In [ ]:
# Garder uniquement quelques années
survey_2000 = surveys_complet[surveys_complet["year"].isin([2000, 2001, 2002])]

(p9.ggplot(data=survey_2000,
           mapping=p9.aes(x='hindfoot_length',
                          y='weight',
                          color='species_id'))
    + p9.geom_point(alpha=0.1)
    + p9.facet_grid('sex ~ year')
)

### Exercice - Facettes
* Créez deux facettes selon le `sex`
* Chaque facette aura :
  * Les années en axe des x
  * Le poids moyen en axe des y
  * Une courbe par espèce

In [ ]:
yearly_weight = surveys_complet.groupby(['year',
                                          'species_id',
                                          'sex'])['weight'].mean().reset_index()
(p9.ggplot(data=yearly_weight,
           mapping=p9.aes(x='year',
                          y='weight',
                          color='species_id'))
    + p9.geom_line()
    + p9.facet_wrap('sex')
)

## Ajustements supplémentaires
* La fonction `theme()` retourne un objet permettant d'orienter verticalement le texte sur l'axe des `x` :

In [ ]:
mon_theme = p9.theme(
    axis_text_x=p9.element_text(angle=90),
    text=p9.element_text(size=10))

(p9.ggplot(data=surveys_complet,
           mapping=p9.aes(x='factor(year)'))
    + p9.geom_bar()
    + p9.xlab("Year")
    + mon_theme
)

## Résumé technique
* **Module Plotnine**
  * `import plotnine as p9`
* **Création d'un graphique vierge** avec Plotnine
  * `p9.ggplot(data=df)`
* **Assigner des variables** à des éléments du graphique
  * `p9.ggplot(data=df, mapping=p9.aes(...))`
    * Exemple : `p9.aes(x='var1', y='var2', color='var3')`
  * Différentes variables :
    * Axes : `x`, `y`, `='factor(var)'`
    * Couleurs : `alpha`, `color`, `colour`, `fill`
    * Formes : `linetype`, `shape`, `size`
* **Ajout d'éléments géométriques** à afficher
  * `p9.geom_point(alpha=0.1, color="green")`
  * `p9.geom_line()`
  * `p9.geom_bar()`, `p9.geom_jitter()`
  * `p9.geom_boxplot()`, `p9.geom_violin()`
* **Configuration des axes, des étiquettes et du thème**
  * `p9.scale_x_log10()` et `p9.scale_y_log10()`
  * `p9.xlab("Axe en X")` et `p9.ylab("Axe en Y")`
  * `p9.ggtitle("Longueur de patte selon le poids")`
  * `p9.theme_bw()`
  * `p9.theme()`
    * `legend_position=` : `top`, `bottom`, `left`, `right`
    * `axis_text_x=p9.element_text(angle=90)`
    * `text=p9.element_text(size=10)`
* **Graphiques multiples**
  * `p9.facet_wrap('varN')`
  * `p9.facet_grid('rangées ~ colonnes')`